In [2]:

# Barebones code to plot interactive correlation plot on web

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
import plotly.io as pio
from IPython.display import display, HTML
from plotly.offline import plot
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import random

# Set plotly renderer for Voila
pio.renderers.default = 'iframe_connected'


# Optional debug logger
def log(msg):
    with open("voila_debug_log.txt", "a", encoding="utf-8") as f:
        f.write(str(msg) + "\n")
        
# Clear debug log
with open("voila_debug_log.txt", "w") as f:
    f.write("Starting new Voila session...\n")



In [3]:
# Load merged dataset
all_merged_data_annually = pd.read_csv('all_merged_data_annually.csv', index_col=0)

# Prepend 'site-agnostic_' to columns without '_'
new_columns = []
for col in all_merged_data_annually.columns:
    if '_' not in col and col != 'depth(cm)':
        new_columns.append(f'site-agnostic_{col}')
    else:
        new_columns.append(col)

all_merged_data_annually.columns = new_columns



FileNotFoundError: [Errno 2] No such file or directory: 'all_merged_data_annually.csv'

In [13]:
# Load raw (non-interpolated) data
raw_data = pd.read_csv("merged_data_MAR_LOI_GS.csv")

# Prepend 'site-agnostic_' to non-underscore columns (same logic as above)
new_columns = []
for col in raw_data.columns:
    if '_' not in col and col != 'depth(cm)':
        new_columns.append(f'site-agnostic_{col}')
    else:
        new_columns.append(col)

raw_data.columns = new_columns


In [5]:
# Build site names
site_names = list(dict.fromkeys(
    col.split('_')[0] for col in all_merged_data_annually.columns
    if '_' in col and col != 'depth(cm)'
))

# Build variables
variables = list(dict.fromkeys(
    col.split('_', 1)[1] for col in all_merged_data_annually.columns
    if '_' in col and col != 'depth(cm)'
))


In [6]:
def random_color():
    return f"rgb({random.randint(0,255)}, {random.randint(0,255)}, {random.randint(0,255)})"

In [24]:
def update_plot(site1, site2, year_range, var1, var2, include_raw):
    try:
        col1 = f"{site1}_{var1}"
        col2 = f"{site2}_{var2}"

        if col1 not in all_merged_data_annually.columns or col2 not in all_merged_data_annually.columns:
            return

        # Prepare data
        df1 = all_merged_data_annually[[col1]].dropna().rename(columns={col1: 'X'}).reset_index()
        df2 = all_merged_data_annually[[col2]].dropna().rename(columns={col2: 'Y'}).reset_index()

        # Build figure
        fig = go.Figure()
        
        color1 = random_color()
        color2 = random_color()

        
        if include_raw:
            # === Raw measured points for site1 ===
            raw_x_col = f"{site1}_{var1}"
            raw_x_year = f"{site1}_Year"
            if raw_x_col in raw_data.columns and raw_x_year in raw_data.columns:
                raw_df1 = raw_data[[raw_x_year, raw_x_col]].dropna()
                fig.add_trace(go.Scatter(
                    x=raw_df1[raw_x_year],
                    y=raw_df1[raw_x_col],
                    mode='markers',
                    name=f'{site1} {var1} (raw)',
                    marker=dict(symbol='circle', size=6, color=color1, opacity=0.8),
                    yaxis='y'
                ))

            # === Raw measured points for site2 ===
            raw_y_col = f"{site2}_{var2}"
            raw_y_year = f"{site2}_Year"
            if raw_y_col in raw_data.columns and raw_y_year in raw_data.columns:
                raw_df2 = raw_data[[raw_y_year, raw_y_col]].dropna()
                fig.add_trace(go.Scatter(
                    x=raw_df2[raw_y_year],
                    y=raw_df2[raw_y_col],
                    mode='markers',
                    name=f'{site2} {var2} (raw)',
                    marker=dict(symbol='x', size=6, color=color2, opacity=0.8),
                    yaxis='y2'
                ))


        # Determine if y-axes should be twinned
        twin_axes = var1 == var2

        # Get global min/max for this variable across both sites if twinned
        if twin_axes:
            y_min = min(df1['X'].min(), df2['Y'].min())
            y_max = max(df1['X'].max(), df2['Y'].max())


        df1_zoom = df1[(df1['Year'] >= year_range[0]) & (df1['Year'] <= year_range[1])]
        df2_zoom = df2[(df2['Year'] >= year_range[0]) & (df2['Year'] <= year_range[1])]

        df_zoom_merged = pd.merge(df1_zoom, df2_zoom, on='Year')

        if len(df_zoom_merged) >= 2 and df_zoom_merged['X'].nunique() > 1 and df_zoom_merged['Y'].nunique() > 1:
            pearson_r, pearson_p = pearsonr(df_zoom_merged['X'], df_zoom_merged['Y'])
            spearman_rho, spearman_p = spearmanr(df_zoom_merged['X'], df_zoom_merged['Y'])

            corr_label = (
                f"Pearson r = {pearson_r:.2f}, p = {pearson_p:.2g}<br>"
                f"Spearman ρ = {spearman_rho:.2f}, p = {spearman_p:.2g}<br>"
                "Correlation used for exploratory purposes only; raw data is annually interpolated."
            )
        else:
            corr_label = "Correlation not valid"




        fig.add_trace(go.Scatter(x=df1['Year'], y=df1['X'],
                                 mode='lines',
                                 name=f'{site1} {var1} (all)',
                                 line=dict(color=color1, dash='dot')))

        fig.add_trace(go.Scatter(x=df2['Year'], y=df2['Y'],
                                 mode='lines',
                                 name=f'{site2} {var2} (all)',
                                 line=dict(color=color2, dash='dot'),
                                 yaxis="y2"))

        fig.add_trace(go.Scatter(x=df1_zoom['Year'], y=df1_zoom['X'],
                                 mode='lines',
                                 name=f'{site1} {var1} (zoom)',
                                 line=dict(color=color1, width=4)))

        fig.add_trace(go.Scatter(x=df2_zoom['Year'], y=df2_zoom['Y'],
                                 mode='lines',
                                 name=f'{site2} {var2} (zoom)',
                                 line=dict(color=color2, width=4),
                                 yaxis="y2"))

        fig.update_layout(
            title=dict(
            text=f"{site1} {var1} vs {site2} {var2} ({year_range[0]}–{year_range[1]})<br>{corr_label}",
            x=0.5,
            xanchor='center',
            y=0.95  # Adjust the vertical position of the title
            ),
            xaxis=dict(title='Year'),
            # Left y-axis (site1)
            yaxis=dict(
                title=dict(text=f'{site1} {var1}', font=dict(color=color1)),
                tickfont=dict(color=color1),
                showgrid=True,
                range=[y_min, y_max] if twin_axes else None
            ),
            # Right y-axis (site2)
            yaxis2=dict(
                title=dict(text=f'{site2} {var2}', font=dict(color=color2)),
                tickfont=dict(color=color2),
                overlaying='y',
                side='right',
                showgrid=False,
                showticklabels=True,
                range=[y_min, y_max] if twin_axes else None
            ),
            legend=dict(x=0.5, y=-0.2, orientation='h', xanchor='center'),
            margin=dict(t=100)

        )


        display(HTML(plot(fig, include_plotlyjs=True, output_type='div')))
        return

    except Exception as e:
        print(f"[ERROR] Exception in update_plot: {e}")
        return


In [1]:
site1_selector = widgets.Dropdown(
    options=site_names,
    value=site_names[0],
    description='Site 1:'
)

site2_selector = widgets.Dropdown(
    options=site_names,
    value=site_names[0],
    description='Site 2:'
)

var1_selector = widgets.Dropdown(
    options=[],
    description='Variable 1:'
)

var2_selector = widgets.Dropdown(
    options=[],
    description='Variable 2:'
)

def update_var1(*args):
    site1 = site1_selector.value
    valid_vars = [v.replace(f"{site1}_", "") for v in all_merged_data_annually.columns if v.startswith(f"{site1}_") and not v.endswith("_Year")]
    current = var1_selector.value
    var1_selector.options = valid_vars
    if current in valid_vars:
        var1_selector.value = current
    elif valid_vars:
        var1_selector.value = valid_vars[0]


def update_var2(*args):
    site2 = site2_selector.value
    valid_vars = [v.replace(f"{site2}_", "") for v in all_merged_data_annually.columns if v.startswith(f"{site2}_") and not v.endswith("_Year")]
    current = var2_selector.value
    var2_selector.options = valid_vars
    if current in valid_vars:
        var2_selector.value = current
    elif valid_vars:
        var2_selector.value = valid_vars[0]


site1_selector.observe(update_var1, names='value')
site2_selector.observe(update_var2, names='value')

update_var1()
update_var2()

year_slider = widgets.IntRangeSlider(
    value=[1980, 2000],
    min=1900,
    max=int(all_merged_data_annually.index.max()),
    step=1,
    description='Year Range:',
    continuous_update=False,
    readout=True,
    readout_format='d'
)
year_slider.layout.width = '600px'  # Optional: make it longer


include_raw_checkbox = widgets.Checkbox(
    value=True,
    description='Include raw points',
    indent=False
)

# === 5. Interactive output ===
ui = widgets.VBox([
    widgets.HBox([site1_selector, var1_selector]),
    widgets.HBox([site2_selector, var2_selector]),
    year_slider,
    include_raw_checkbox  # <-- Add this
])

out = widgets.interactive_output(
    update_plot,
    {
        'site1': site1_selector,
        'site2': site2_selector,
        'year_range': year_slider,
        'var1': var1_selector,
        'var2': var2_selector,
        'include_raw': include_raw_checkbox
    }
)

display(ui, out)

NameError: name 'widgets' is not defined